In [ ]:
file_args_key = "live"
proof_number = 0
save_name_pth = "live_3_8_0_30_0.5_write_08_25_2023_14_26_10_dict.pth"
yesorno_inputdim = 50  
out_pth = r"D:\out_pth" 
save_name_pth = out_pth + "\\" + save_name_pth


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets

from sklearn.model_selection import KFold
import copy
from tqdm import tqdm
import numpy as np

from model_nn import RNN, RNN50
# from model_nn_min import RNN, RNN_gradient,RNN50, RNN50_gradient
from args_fvg import args_parser_live, args_parser_shake

from nltk.tag import pos_tag
import re
import time


if file_args_key == "shake":
    args = args_parser_shake()
elif file_args_key == "live":
    args = args_parser_live()

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

if file_args_key == "shake":
    train_data.examples = train_data.examples[:len(train_data.examples)//125]
    test_data.examples = test_data.examples[:len(test_data.examples)//125]

len_train_data = len(train_data)
len_test_data = len(test_data)

print('len of train data:', len(train_data))
print('len of test data:', len(test_data))
print(test_data.examples[1].text) 
print(test_data.examples[1].label)

# word2vec, glove

if yesorno_inputdim == 100:
    TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.100d')
if yesorno_inputdim == 50:
    TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.50d')

LABEL.build_vocab(train_data)

# TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.50d')
# LABEL.build_vocab(train_data)

# TEXT.build_vocab(test_data, max_size=10000, vectors='glove.6B.100d')
# LABEL.build_vocab(test_data)

vocab_size =  len(TEXT.vocab)
pretrained_embedding = TEXT.vocab.vectors
print(vocab_size, pretrained_embedding.shape)

def RFFM(i, x):
    torch.manual_seed(i)
    input_dim = 100
    output_dim = 50
    sigma = 1.0
    omega = torch.randn(output_dim, input_dim) * sigma
    b = torch.rand(output_dim) * 2 * np.pi
    x = x.view(-1, input_dim)
    z = torch.cos(torch.mm(x, omega.t()) + b)
    Z_Out =  torch.sqrt(2.0 / torch.tensor(output_dim)) * z
    return Z_Out

if yesorno_inputdim == 50:
    model = RNN50(vocab_size, args, name='server').to(args.device)
    model.embedding.weight.data.copy_(pretrained_embedding)

elif (proof_number == 1) or (proof_number == 3) or (proof_number == 6):  
    new_vectors = []
    for i, word in enumerate(TEXT.vocab.itos):
        old_vector = pretrained_embedding[i]
        new_vector = RFFM(i, old_vector)
        new_vectors.append(new_vector)
    new_pretrained_embedding = torch.stack(new_vectors, dim=0)
    new_pretrained_embedding = new_pretrained_embedding.reshape(-1,50)
    print(new_pretrained_embedding.shape)
    
    model = RNN50(vocab_size, args, name='server').to(args.device)   # 
    model.embedding.weight.data.copy_(new_pretrained_embedding)
            
elif (proof_number == 2) or (proof_number == 0) :
    model = RNN(vocab_size, args, name='server').to(args.device)
    model.embedding.weight.data.copy_(pretrained_embedding)
    


['Actor', 'turned', 'director', 'Bill', 'Paxton', 。。。
pos

In [ ]:
model.load_state_dict(torch.load(save_name_pth))
model.eval()  

In [ ]:

train_indices = list(range(len(train_data.examples)))
k = args.K 
random_seed = 42  
kfold = KFold(n_splits=k, shuffle=True, random_state=random_seed)
train_subsets = []
for _, train_idx in kfold.split(train_indices):
    train_subset = data.Dataset([train_data.examples[i] for i in train_idx], fields=[('text', TEXT), ('label', LABEL)])
    # print(len(train_subset))
    train_subsets.append(train_subset)

test_indices = list(range(len(test_data.examples)))
random_seed = 24 
kfold_te = KFold(n_splits=k, shuffle=True, random_state=random_seed)
test_subsets = []
for _, test_idx in kfold_te.split(test_indices):
    test_subset = data.Dataset([test_data.examples[i] for i in test_idx], fields=[('text', TEXT), ('label', LABEL)])
    # print(len(test_subset))
    test_subsets.append(test_subset)


def iterator(tri_data, tes_data):
    train_iterator, test_iterator = data.BucketIterator.splits(
        (tri_data, tes_data),
        batch_size = args.B,
        sort_key = lambda x: len(x.text),
        sort_within_batch=True,
        device=args.device
    )
    return train_iterator, test_iterator

train_all_sub = []
testa_all_sub = []

for index in range(len(train_subsets)) :
    data_i, data_j = train_subsets[index ], test_subsets[index ]
    data_i_reu, data_j_reu = iterator(data_i, data_j)
    train_all_sub.append(data_i_reu)
    testa_all_sub.append(data_j_reu)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

loss_function = nn.BCEWithLogitsLoss().to(args.device) 

def binary_acc(preds, y):
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc

def test(args, model, k):
    model.eval()
    Val = testa_all_sub[k] 
    avg_acc = []
    avg_auc = []
    avg_ks = []
    avg_precision = []
    avg_recall = []
    avg_f1 = []
    
    with torch.no_grad():
        for seq in Val:
            y_pred = model(seq.text).squeeze(1)
            acc = binary_acc(y_pred, seq.label).item()
            avg_acc.append(acc)
            auc = roc_auc_score(seq.label.cpu(), torch.sigmoid(y_pred).cpu())
            avg_auc.append(auc)
            fpr, tpr, thresholds = roc_curve(seq.label.cpu(), torch.sigmoid(y_pred).cpu())
            ks = np.max(np.abs(tpr - fpr))
            avg_ks.append(ks)
            y_pred_binary = torch.round(torch.sigmoid(y_pred)).cpu().numpy()
            precision = precision_score(seq.label.cpu(), y_pred_binary)
            recall = recall_score(seq.label.cpu(), y_pred_binary)
            f1 = f1_score(seq.label.cpu(), y_pred_binary)
            avg_precision.append(precision)
            avg_recall.append(recall)
            avg_f1.append(f1)
            
    avg_acc = np.mean(avg_acc)
    avg_auc = np.mean(avg_auc)
    avg_ks = np.mean(avg_ks)
    avg_precision = np.mean(avg_precision)
    avg_recall = np.mean(avg_recall)
    avg_f1 = np.mean(avg_f1)
    
    # print('客户端 %s avg acc:' % k, avg_acc)
    # print('客户端 %s avg auc:' % k, avg_auc)
    # print('客户端 %s avg ks:' % k, avg_ks)
    # print('客户端 %s avg precision:' % k, avg_precision)
    # print('客户端 %s avg recall:' % k, avg_recall)
    # print('客户端 %s avg f1:' % k, avg_f1)
    # print("\n")

    print(f'客户端 {k} avg acc: {avg_acc}, avg auc: {avg_auc}')

# test(args, model, k)

In [ ]:
for k in range(len(args.clients)):
    with torch.no_grad():
        test(args, model, k)